In [91]:
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader('docs//health supplements//1. dietary supplements - for whom.pdf')

In [92]:
docs = loader.load()
len(docs)

17

In [93]:
print(docs[0].page_content)

International  Journal  of
Environmental Research
and Public Health
Review
Dietary Supplements—For Whom? The Current State of
Knowledge about the Health Effects of Selected
Supplement Use
Regina Ewa Wierzejska


Citation: Wierzejska, R.E. Dietary
Supplements—For Whom? The
Current State of Knowledge about the
Health Effects of Selected Supplement
Use. Int. J. Environ. Res. Public Health
2021, 18, 8897. https://doi.org/
10.3390/ijerph18178897
Academic Editor: Paul B. Tchounwou
Received: 15 July 2021
Accepted: 21 August 2021
Published: 24 August 2021
Publisher’s Note: MDPI stays neutral
with regard to jurisdictional claims in
published maps and institutional afﬁl-
iations.
Copyright: © 2021 by the author.
Licensee MDPI, Basel, Switzerland.
This article is an open access article
distributed
under
the
terms
and
conditions of the Creative Commons
Attribution (CC BY) license (https://
creativecommons.org/licenses/by/
4.0/).
Department of Nutrition and Nutritional Value of Food,

In [94]:
print(docs[0].metadata)

{'source': 'docs//health supplements//1. dietary supplements - for whom.pdf', 'file_path': 'docs//health supplements//1. dietary supplements - for whom.pdf', 'page': 0, 'total_pages': 17, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'iLovePDF', 'creationDate': '', 'modDate': 'D:20241021113754Z', 'trapped': ''}


In [95]:
import os

pdfs = []

for root, dirs, files in os.walk('docs'):
    for file in files:
        if file.endswith('.pdf'):
            pdfs.append(os.path.join(root, file))

len(pdfs)

5

In [96]:
docs = []

for pdf in pdfs:
    loader = PyMuPDFLoader(pdf)
    temp = loader.load()
    docs.extend(temp)

len(docs)

64

In [97]:
def format_text(docs):
    return '\n\n'.join([x.page_content for x in docs])

context = format_text(docs)
len(context)

258804

In [98]:
os.path.join(os.path.expanduser("~"), ".cache", "huggingface", "hub") == "C:\\Users\\leand\\.cache\\huggingface\\hub"

True

In [99]:
import os

# Get the cache directory
cache_dir = os.getenv("TRANSFORMERS_CACHE", os.path.join(os.path.expanduser("~"), ".cache", "huggingface", "hub"))

# List all model folders in the cache directory
if os.path.exists(cache_dir):
    models = [name for name in os.listdir(cache_dir) if os.path.isdir(os.path.join(cache_dir, name))]
    print("Downloaded Hugging Face Models:")
    for model in models:
        print(f"- {model}")
else:
    print("No models found in the Hugging Face cache directory.")


Downloaded Hugging Face Models:
- .locks
- datasets--mrdbourke--learn_hf_food_not_food_image_captions
- models--CIDAS--clipseg-rd64-refined
- models--CompVis--stable-diffusion-v1-4
- models--dbmdz--bert-large-cased-finetuned-conll03-english
- models--distilbert--distilbert-base-cased
- models--distilbert--distilbert-base-cased-distilled-squad
- models--distilbert--distilbert-base-uncased-finetuned-sst-2-english
- models--distilbert--distilroberta-base
- models--facebook--bart-large-mnli
- models--Helsinki-NLP--opus-mt-en-es
- models--lllyasviel--Annotators
- models--openai-community--gpt2
- models--RunDiffusion--RunDiffusion-XL-Beta
- models--runwayml--stable-diffusion-v1-5
- models--sshleifer--distilbart-cnn-12-6
- models--stabilityai--stable-diffusion-2-1
- models--stablediffusionapi--deliberate-v2
- models--Systran--faster-whisper-large-v3
- models--takuma104--control_v11
- models--thibaud--controlnet-openpose-sdxl-1.0


In [100]:
import tiktoken

encoding = tiktoken.encoding_for_model('gpt-4o-mini')

In [101]:
encoding.encode('congratulations')

[542, 111291, 14571]

In [102]:
len(encoding.encode(docs[0].page_content))

969

In [103]:
len(encoding.encode(context))

60271

The size is supported by llama3 thatr supports 128k

In [104]:
969*64

62016

In [105]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import textwrap

In [106]:
base_url = "http://localhost:11434"
model = "llama3.2:3b"

llm = ChatOllama(base_url=base_url,
                 model=model,
                 temperature=0.1,
                 num_predict=256
                 )

In [107]:
system = SystemMessagePromptTemplate.from_template("""
You are a helpful AI assistant who answer user question based on provide context.
Do not answer in more than {words} words
""")

In [108]:
prompt = """Answer user question based on the provided context ONLY! if do not know the answer, just sai "I don't know".
### Context:
{context}

### Question:
{question}

### Answer:
"""

prompt = HumanMessagePromptTemplate.from_template(prompt)

messages = [system, prompt]
template = ChatPromptTemplate(messages)

In [109]:
# template.invoke({'context':context, 'question': 'How do I gain mass>', 'words': 50})

In [110]:
qna_chain = template | llm | StrOutputParser()
qna_chain

ChatPromptTemplate(input_variables=['context', 'question', 'words'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['words'], input_types={}, partial_variables={}, template='\nYou are a helpful AI assistant who answer user question based on provide context.\nDo not answer in more than {words} words\n'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Answer user question based on the provided context ONLY! if do not know the answer, just sai "I don\'t know".\n### Context:\n{context}\n\n### Question:\n{question}\n\n### Answer:\n'), additional_kwargs={})])
| ChatOllama(model='llama3.2:3b', num_predict=256, temperature=0.1, base_url='http://localhost:11434')
| StrOutputParser()

In [111]:
response = qna_chain.invoke({'context':context, 'question': 'How to gain muscle mass?', 'words': 50})
print(textwrap.fill(response))

I can't provide a direct answer to your question as it seems unrelated
to the provided text about botanical supplements and their potential
toxic mechanisms. However, I can offer some general information on how
to gain muscle mass.  Gaining muscle mass requires a combination of
proper nutrition, consistent training, and sufficient rest. Here are
some tips:  1. **Eat enough protein**: Protein is essential for muscle
growth and repair. Aim for 1-1.5 grams of protein per kilogram of body
weight daily. 2. **Increase caloric intake**: To build muscle, you
need to be in a calorie surplus, meaning you consume more calories
than your body burns. This will provide your body with the necessary
energy to build new muscle tissue. 3. **Focus on compound exercises**:
Compound exercises like squats, deadlifts, bench press, and rows work
multiple muscle groups at once and are effective for building muscle.
4. **Progressive overload**: Gradually increase the weight or
resistance you're lifting over tim

In [112]:
response = qna_chain.invoke({'context':context, 'question': 'How to reduce the weight?', 'words': 50})
print(textwrap.fill(response))

There is no answer provided for the question "How to reduce the
weight?" as it seems unrelated to the text about botanical supplements
and their potential toxic mechanisms. However, I can provide some
general information on how to reduce weight.  To reduce weight, a
combination of healthy diet, regular exercise, and lifestyle changes
can be effective. Here are some tips:  1. Eat a balanced diet: Focus
on whole, unprocessed foods like vegetables, fruits, whole grains,
lean proteins, and healthy fats. 2. Keep track of calorie intake: Use
a food diary or an app to monitor your daily calorie consumption. 3.
Increase physical activity: Aim for at least 150 minutes of moderate-
intensity exercise or 75 minutes of vigorous-intensity exercise per
week. 4. Stay hydrated: Drink plenty of water throughout the day. 5.
Get enough sleep: Aim for 7-9 hours of sleep per night. 6. Manage
stress: Engage in stress-reducing activities like meditation, yoga, or
deep breathing exercises.  It's also importan

In [113]:
response = qna_chain.invoke({'context':context, 'question': 'How to do weight loss?', 'words': 50})
print(textwrap.fill(response))

I can't answer that question as it is not related to the text about
botanical supplements and their potential toxic mechanisms. If you
have a different question or need information on a specific topic,
feel free to ask!


In [114]:
response = qna_chain.invoke({'context':context, 'question': 'How to live in planet earth?', 'words': 50})
print(textwrap.fill(response))

It seems like there was a mistake. The text you provided doesn't seem
to be related to the question "How to live in planet earth?" at all.
It appears to be a passage about botanical supplements and their
potential toxic mechanisms.  If you could provide more context or
clarify what you're looking for, I'd be happy to try and assist you.


# Summarization

In [118]:
system = SystemMessagePromptTemplate.from_template("""
You are a helpful AI assistant who work as document summarizer.
You muast not hallucinate or provide any false information
""")

prompt = """Summarize the given context in {words}".
### Context:
{context}

### Summary:
"""

prompt = HumanMessagePromptTemplate.from_template(prompt)

messages = [system, prompt]
template = ChatPromptTemplate(messages)

summary_chain = template | llm | StrOutputParser()

In [119]:
summary = summary_chain.invoke({'context':context, 'words': 50})
print(textwrap.fill(summary))

This article discusses the potential therapeutic and toxic mechanisms
of action of various botanical supplements, including their primary
active constituents, typical use and dosage, and reported adverse
effects. The authors highlight that concurrent exposure to other
compounds and the heterogeneity of herbal supplements can make it
difficult to determine toxic mechanisms in clinical cases.  The
article reviews case reports of adverse effects associated with
various botanicals, including:  * Black cohosh: jaundice and liver
failure * Kava kava: liver toxicity, depletion of glutathione, and
inhibition of cyclooxygenases * Saw palmetto: cholestatic hepatitis
and pancreatitis * Echinacea: acute liver failure * Valerian: jaundice
that was reversed by steroid administration * Yohimbe: sympathomimetic
properties leading to seizures and hypertension * Milk thistle:
exacerbation of hemochromatosis * Ginseng: transient ischemia attack *
Black cohosh: bradycardia  The authors also discuss potent

# Report Generation from PDF Document

In [121]:
response = qna_chain.invoke({'context':context, 'question': 'Provide a detailed reposrt fom the provided context, write the answer in MarkDown', 'words': 2000})
print(response)

**Botanical Supplements: A Review of Their Primary Active Constituents, Typical Use and Dosage, and Reported Adverse Effects**

The use of botanical supplements has become increasingly popular, with many individuals seeking to incorporate natural remedies into their health regimens. However, the safety and efficacy of these supplements can be compromised by various factors, including concurrent exposure to other compounds, heterogeneity of herbal supplements, and inadequate reporting of adverse effects.

**Commonly Used Botanical Supplements**

1. **Black Cohosh (Cimicifuga racemosa)**
	* Primary active constituents: Cymaritenol A, Cymaritenol B, and others
	* Typical use and dosage: Menopausal symptoms, hot flashes, and night sweats
	* Reported adverse effects: Jaundice, liver failure, and pathological oxidative stress
2. **Kava Kava**
	* Primary active constituents: Kavain, Dihydrokavain, and others
	* Typical use and dosage: Anxiety, insomnia, and restlessness
	* Reported adverse ef